In [12]:
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Vectorize Function

This function takes in a FEN string and outputs 13 8x8 arrays to represent the FEN string as bits

In [13]:
def vectorize(fen):
    data = re.split(" ", fen)
    rows= re.split("/", data[0])
    turn = data[1]
    can_castle = data[2]
    passant = data[3]
    half_moves = data[4]
    full_moves = data[5]
    
    bit_vector = np.zeros((13, 8, 8), dtype=np.uint8)
    #what layer each piece is found on
    piece_to_layer = {
            'R': 1,
            'N': 2,
            'B': 3,
            'Q': 4,
            'K': 5,
            'P': 6,
            'p': 7,
            'k': 8,
            'q': 9,
            'b': 10,
            'n': 11,
            'r': 12
        }
    #find each piece based on type
    for r,value in enumerate(rows):
        colum = 0
        for piece in value:
            if piece in piece_to_layer:
                bit_vector[piece_to_layer[piece],r,colum] =1
                colum += 1
            else:
                colum += int(piece)
    
    if turn.lower() == 'w':
        bit_vector [0,7,4] =1
    else:
        bit_vector [0,0,4] =1
        
    #where each castle bit is located
    castle ={
        'k': (0,0),
        'q': (0,7),
        'K': (7,0),
        'Q': (7,7),
        }

    for value in can_castle:
        if value in castle:
            bit_vector[0,castle[value][0],castle[value][1]] = 1
    
    #put en-passant square in the vector
    if passant != '-':
        bit_vector[0,  5 if (int(passant[1])-1 == 3) else 2 , ord(passant[0]) - 97,] = 1
        
    return bit_vector

# print(vectorize(r'rnbqkb1r/ppp1pppp/3p4/3nP3/3P4/8/PPPK1PPP/RNBQ1BNR b kq - 1 4')[0])
# print(vectorize(r'r3r1k1/pppq2bp/3pp1p1/3P4/2PpP3/6P1/PP1N2K1/R1BQR3 b - e3 0 19')[0])

In [14]:
# Expects data frame with 1 column "FEN", and another "Evaluation"
def split_and_reshape(x, y, random, train_size):
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=random, train_size = train_size)
    
    # reshape so we can enter this into the model
    x_train = x_train.reshape(len(x_train), 832)
    x_test = x_test.reshape(len(x_test), 832)

    return x_train, x_test, y_train, y_test

In [15]:
def getModel():
    # Define the neural network architecture
    model = tf.keras.Sequential([
        #tf.keras.layers.Flatten(input_shape=(13, 8, 8)),
        tf.keras.layers.Dense(832, activation='relu'),
        tf.keras.layers.Dense(400, activation='relu'),
        tf.keras.layers.Dense(200, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
    ])

    # Compile the model
    model.compile(optimizer='RMSprop',
        loss='MeanSquaredError',
        metrics=['accuracy'])
    
    return model

In [16]:
def train_neural_network(FEN, Evaluation):
    # Define the neural network architecture    
    model = getModel()

    ## Get training and testing data
    x_train, x_test, y_train, y_test = split_and_reshape(FEN, Evaluation, random=0, train_size = .75)

    # Train the model
    model.fit(x_train, y_train, epochs=10, batch_size=128)

    # Evaluate the model on the test data
    loss, accuracy = model.evaluate(x_test, y_test)

    # Print the evaluation metrics
    print(f"Test loss: {loss:.4f}")
    print(f"Test accuracy: {accuracy:.4f}")

    return model

## Handling Weird Evaluation Scores

Some are like "#+6", or even stranger like "\ufeff23". Want them all to be +/- integers

In [17]:
# This is to handle weird cases in Evaluation such as "#+6", or "\ufeff23"
def parse_evaluation(value):
    if isinstance(value, str):
        value = re.sub(r'[^0-9+\-]', '', value)  # Remove anything that's not a digit or '+' or '-'
        return int(value.replace('-', '')) * (-1 if '-' in value else 1)
    return value

# Apply the parsing function to the 'Evaluation' column
# dfcopy['Evaluation'] =  dfcopy['Evaluation'].apply(parse_evaluation)

In [18]:
def preprocess_scores(scores):
    #sets all scores within -150 to 150
    #mate scores are set to +-151
    for i, score in enumerate(scores):
        if score[0:2] == '#+':
            scores[i] = 151
        elif score[0:2] == '#-':
            scores[i] = -151
        elif int(score) > 150:
            scores[i] = 150
        elif int(score) < -150:
            scores[i] = -150
        else:
            scores[i] = int(score)
    #scale between 0 and 1
    scores += 150
    scores /= 300
    scores = scores.astype('float32')
    return scores

# Try making the neural network

Hasn't worked so far

In [25]:
nrows = 50_000

df = pd.read_csv('../data/kaggleDataset/chessData.csv', nrows=nrows, dtype={'FEN':str, 'Evaluation':str})

positions = np.ndarray(shape=(nrows,13,8,8))
for i, f in enumerate(df.FEN):
    positions[i] = vectorize(f)

scores = preprocess_scores(df.Evaluation.to_numpy())

x_train, x_test, y_train, y_test = train_test_split(positions, scores, random_state=0, train_size = .75)


model = train_neural_network(positions, scores)

Epoch 1/10
293/293 [==============================] - 5s 13ms/step - loss: 0.0850 - accuracy: 0.2746
Epoch 2/10
293/293 [==============================] - 5s 16ms/step - loss: 0.0255 - accuracy: 0.3190
Epoch 3/10
293/293 [==============================] - 5s 18ms/step - loss: 0.0155 - accuracy: 0.3211
Epoch 4/10
293/293 [==============================] - 6s 21ms/step - loss: 0.0113 - accuracy: 0.3216
Epoch 5/10
293/293 [==============================] - 6s 20ms/step - loss: 0.0085 - accuracy: 0.3219
Epoch 6/10
293/293 [==============================] - 4s 13ms/step - loss: 0.0068 - accuracy: 0.3220
Epoch 7/10
293/293 [==============================] - 4s 14ms/step - loss: 0.0057 - accuracy: 0.3219
Epoch 8/10
293/293 [==============================] - 4s 13ms/step - loss: 0.0049 - accuracy: 0.3220
Epoch 9/10
293/293 [==============================] - 4s 13ms/step - loss: 0.0042 - accuracy: 0.3221
Epoch 10/10
391/391 [==============================] - 2s 4ms/step - loss: 0.0122 - accurac